In [1]:
import pandas as pd, numpy as np, dataprep as dp
import torch

In [2]:
df = pd.read_csv("/Users/florian/Documents/github/study/IoT/IoT/aggregated_hourly.csv")
a0 = ["hka-aqm-a017", "hka-aqm-a014"]
a1 = ["hka-aqm-a101", "hka-aqm-a102", "hka-aqm-a103", "hka-aqm-a106", "hka-aqm-a107", "hka-aqm-a108", "hka-aqm-a111", "hka-aqm-a112"]

In [3]:
df_cutoff = df.copy
df_cutoff = dp.cutoff_data(df, "2023-01-13", "2023-03-28")
df_mean = dp.build_lvl_df(df_cutoff, a1, ["tmp", "hum", "CO2", "VOC"], reset_ind= True)
df_mean.head()

,tmp,hum,CO2,VOC
0,23.0950,34.000,573.0,754.0
1,23.1125,33.840,545.0,750.0
2,23.0950,33.865,532.0,741.0
3,23.1650,33.825,516.0,754.0
4,23.2725,33.780,507.0,767.0


In [4]:
X = df_mean.to_numpy()
y = df_mean["tmp"].shift(-1).to_numpy()
X.shape, y.shape

((1775, 4), (1775,))

In [5]:
X_train, X_test, y_train, y_test = dp.train_test_split(X, y)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

(torch.Size([1748, 4]),
 torch.Size([1748]),
 torch.Size([27, 4]),
 torch.Size([27]))

In [7]:
X_train_new, X_test_new = dp.format_tensor(X_train), dp.format_tensor(X_test)
X_train = X_train_new
X_test = X_test_new
y_train = y_train[1:]
y_test = y_test[1:]
X_train.shape, X_test.shape, y_train.shape, y_test.shape

(torch.Size([1747, 48, 4]),
 torch.Size([26, 48, 4]),
 torch.Size([1747]),
 torch.Size([26]))

In [8]:
from torch.utils.data import DataLoader, TensorDataset

batch_size = 64

train_data = TensorDataset(X_train, y_train)
test_data = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)